In [5]:
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics

%matplotlib inline

In [7]:
df_football_midfield_Attacks_position_cleaned = pd.read_csv(r"C:\Users\foooo\Datascincecamp\Usecase-7\df_football_midfield_Attacks_position_cleaned.csv")

In [11]:
df_football_midfield_Attacks_position_cleaned.shape

(5006, 21)

In [13]:
df_football_midfield_Attacks_position_cleaned.head(2)

,Unnamed: 0,team,position,height,age,appearance,goals,assists,yellow cards,second yellow cards,...,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded,winger
0,16,Manchester United,midfield-CentralMidfield,182.0,31,55,0.067214,0.313667,0.089619,0.0,...,0.0,0.0,4017,0,0,10,25000000,100000000,3,0
1,17,Manchester United,midfield-CentralMidfield,177.0,29,74,0.144046,0.086428,0.230474,0.0,...,0.0,0.0,3124,280,35,8,20000000,42000000,3,0


In [15]:
df_football_midfield_Attacks_position_cleaned.columns

Index(['Unnamed: 0', 'team', 'position', 'height', 'age', 'appearance',
       'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
       'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
       'games_injured', 'award', 'current_value', 'highest_value',
       'position_encoded', 'winger'],
      dtype='object')

In [17]:
df_football_midfield_Attacks_position_cleaned.drop('Unnamed: 0',axis=1, inplace= True)

In [21]:
df_football_midfield_Attacks_position_cleaned.dtypes

team                    object
position                object
height                 float64
age                      int64
appearance               int64
goals                  float64
assists                float64
yellow cards           float64
second yellow cards    float64
red cards              float64
goals conceded         float64
clean sheets           float64
minutes played           int64
days_injured             int64
games_injured            int64
award                    int64
current_value            int64
highest_value            int64
position_encoded         int64
winger                   int64
dtype: object

In [23]:
categorical_features = ['team' ,'position']
numeric_features = ['height', 'age', 'appearance',
       'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
       'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
       'games_injured', 'award', 'current_value', 'highest_value',
       'position_encoded', 'winger']

In [25]:
df_football_midfield_Attacks_position_cleaned = pd.get_dummies(df_football_midfield_Attacks_position_cleaned, columns=categorical_features)

In [31]:
df_football_midfield_Attacks_position_cleaned.head(1)

,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,...,position_Attack,position_Attack Centre-Forward,position_Attack-LeftWinger,position_Attack-RightWinger,position_Attack-SecondStriker,position_midfield,position_midfield-AttackingMidfield,position_midfield-CentralMidfield,position_midfield-LeftMidfield,position_midfield-RightMidfield
0,182.0,31,55,0.067214,0.313667,0.089619,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,False


In [33]:
correlation = df_football_midfield_Attacks_position_cleaned.corr()
print(correlation['current_value'].sort_values(ascending=False))

current_value                      1.000000
highest_value                      0.826811
minutes played                     0.439847
appearance                         0.410255
award                              0.317545
                                     ...   
position_midfield-RightMidfield   -0.036068
position_Attack                   -0.041045
position_midfield                 -0.049038
goals conceded                          NaN
clean sheets                            NaN
Name: current_value, Length: 402, dtype: float64


In [35]:
threshold = 0.3  # You can change this value based on your requirement

# Filter the correlations
# We use `abs()` for absolute value to consider both strong positive and negative correlations
selected_features = correlation[abs(correlation['current_value']) > threshold]['current_value'].index
selected_features

Index(['appearance', 'minutes played', 'award', 'current_value',
       'highest_value'],
      dtype='object')

In [37]:
selected_features = ['appearance', 'minutes played', 'award', 'current_value',
       'highest_value']

In [39]:
df_football_midfield_Attacks_position_cleaned = df_football_midfield_Attacks_position_cleaned[selected_features]
df_football_midfield_Attacks_position_cleaned.head()

,appearance,minutes played,award,current_value,highest_value
0,55,4017,10,25000000,100000000
1,74,3124,8,20000000,42000000
2,92,4869,11,20000000,50000000
3,31,1157,6,17000000,55000000
4,39,2788,1,800000,800000


In [43]:
X = df_football_midfield_Attacks_position_cleaned.drop(['current_value'], axis=1)
y = df_football_midfield_Attacks_position_cleaned['current_value']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=42)

# sacle the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [45]:
model = LinearRegression()

In [47]:
model.fit(X_train_scaled, y_train)

LinearRegression()

In [49]:
y_pred = model.predict(X_test_scaled)

In [51]:
model.coef_

array([-1928361.05317977,  3045488.8340621 , -2261987.84196354,
        9609966.78711461])

In [53]:
coeff_df = pd.DataFrame(model.coef_,X.columns,columns=['Coefficient'])
coeff_df

,Coefficient
appearance,-1.928361e+06
minutes played,3.045489e+06
award,-2.261988e+06
highest_value,9.609967e+06


In [55]:
print(model.intercept_) 

4279644.105894106
